In [ ]:

def compound_process(t, B, x_cdf, cdf):
    theta_t = np.interp(t, x_cdf, cdf)
    X = np.interp(theta_t, np.linspace(0, 1, len(B)), B)
    return X

def lognormal_multiplier_generator(mu=-0.1, sigma=0.3):
    """
    Returns a function that generates lognormal multipliers with mean 1.

    Parameters:
    - mu: mean of the underlying normal distribution
    - sigma: standard deviation of the underlying normal distribution

    Returns:
    - generator function for multipliers
    """
    # For lognormal to have mean 1, we need to adjust mu
    adjusted_mu = -sigma**2/2

    def generator():
        return np.random.lognormal(adjusted_mu, sigma)

    return generator

def binomial_multiplier_generator(m0, m1, p0=0.5):
    """
    Returns a function that generates binomial multipliers.

    Parameters:
    - m0, m1: multiplier values
    - p0: probability of m0

    Returns:
    - generator function for multipliers
    """
    def generator():
        return m0 if np.random.random() < p0 else m1

    return generator

def simulate_multifractal_model(T=1.0, n_points=1000, frequencies=None, multiplier_generator=None, seed=None):
    """
    Simulate the complete multifractal model with more options.

    Parameters:
    - T: time horizon
    - n_points: number of time steps
    - frequencies: list of frequencies for each component (if None, uses geometric series)
    - multiplier_generator: function to generate multipliers (if None, uses lognormal)
    - seed: random seed

    Returns:
    - t: time points
    - X: log-price process
    - theta: trading time function
    - B: Brownian motion
    """
    if seed is not None:
        np.random.seed(seed)

    # Default frequencies: geometric progression
    if frequencies is None:
        base_freq = 5
        k_levels = 10
        frequencies = [base_freq * 2**i for i in range(k_levels)]

    # Default multiplier generator
    if multiplier_generator is None:
        multiplier_generator = lognormal_multiplier_generator()

    # Generate multifractal measure and trading time
    x_mf, measure, theta = generate_multifractal_measure(
        T, frequencies, multiplier_generator, seed=seed)

    # Generate Brownian motion
    t_bm, B = generate_brownian_motion(T=1.0, N=n_points, seed=seed)

    # Compute compound process
    X = compound_process(t_bm, B, x_mf, theta)

    return t_bm, X, theta, B

def plot_model_components(t, X, x_theta, theta, t_bm, B, measure, save_path=None):
    """
    Plot all components of the multifractal model with improved visualization.

    Parameters:
    - t: time points for log-price
    - X: log-price process
    - x_theta: points where trading time is defined
    - theta: trading time values
    - t_bm: time points for Brownian motion
    - B: Brownian motion values
    - measure: multifractal measure
    - save_path: path to save the figure (if None, figure is displayed)
    """
    plt.figure(figsize=(15, 15))

    # Plot the multifractal measure
    plt.subplot(3, 2, 1)
    plt.plot(x_theta[:-1], measure, 'b-', drawstyle='steps-post')
    plt.xlabel('t', fontsize=12)
    plt.ylabel('Measure μ(t)', fontsize=12)
    plt.title('Multifractal Measure', fontsize=14)
    plt.grid(True, alpha=0.3)

    # Plot the trading time θ(t) (CDF of measure)
    plt.subplot(3, 2, 2)
    plt.plot(x_theta, theta, 'r-')
    plt.xlabel('t', fontsize=12)
    plt.ylabel('θ(t)', fontsize=12)
    plt.title('Trading Time (CDF of Multifractal)', fontsize=14)
    plt.grid(True, alpha=0.3)

    # Plot Brownian motion
    plt.subplot(3, 2, 3)
    plt.plot(t_bm, B, 'g-')
    plt.xlabel('t', fontsize=12)
    plt.ylabel('B(t)', fontsize=12)
    plt.title('Brownian Motion', fontsize=14)
    plt.grid(True, alpha=0.3)

    # Plot the log-price process
    plt.subplot(3, 2, 4)
    plt.plot(t, X, 'purple')
    plt.xlabel('t', fontsize=12)
    plt.ylabel('X(t) = B(θ(t))', fontsize=12)
    plt.title('Log-Price Process (Compound Process)', fontsize=14)
    plt.grid(True, alpha=0.3)

    # Plot increments (returns)
    plt.subplot(3, 2, 5)
    returns = np.diff(X)
    plt.plot(t[1:], returns, color='blue')
    plt.xlabel('t', fontsize=12)
    plt.ylabel('ΔX(t)', fontsize=12)
    plt.title('Log-Price Increments (Returns)', fontsize=14)
    plt.grid(True, alpha=0.3)

    # Plot squared returns (volatility)
    plt.subplot(3, 2, 6)
    squared_returns = returns**2
    plt.plot(t[1:], squared_returns, color='red')
    plt.xlabel('t', fontsize=12)
    plt.ylabel('(ΔX(t))²', fontsize=12)
    plt.title('Squared Returns (Volatility)', fontsize=14)
    plt.grid(True, alpha=0.3)

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    else:
        plt.show()

def analyze_scaling_properties(t, X, q_values=[1, 2, 3, 5], n_scales=20, plot=True):
    """
    Analyze the scaling properties of the returns.

    Parameters:
    - t: time points
    - X: log-price process
    - q_values: list of moment orders to analyze
    - n_scales: number of time scales to analyze
    - plot: whether to plot the results

    Returns:
    - scales: time scales analyzed
    - scaling_exponents: estimated scaling exponents for each q
    """
    dt = t[1] - t[0]
    max_scale = len(t) // 4

    # Define time scales (logarithmically spaced)
    scales = np.unique(np.logspace(0, np.log10(max_scale), n_scales).astype(int))
    moments = np.zeros((len(q_values), len(scales)))

    # Compute moments at different time scales
    for i, scale in enumerate(scales):
        # Get returns at this scale
        returns = X[scale:] - X[:-scale]

        # Compute moments
        for j, q in enumerate(q_values):
            moments[j, i] = np.mean(np.abs(returns)**q)

    # Estimate scaling exponents using log-log regression
    log_scales = np.log(scales * dt)
    log_moments = np.log(moments)

    scaling_exponents = np.zeros(len(q_values))
    for j, q in enumerate(q_values):
        # Linear regression on log-log plot
        coeffs = np.polyfit(log_scales, log_moments[j], 1)
        scaling_exponents[j] = coeffs[0]

    if plot:
        plt.figure(figsize=(12, 8))

        # Plot moments vs. scales
        for j, q in enumerate(q_values):
            plt.plot(log_scales, log_moments[j], 'o-', label=f'q={q}')

            # Plot regression line
            y_fit = scaling_exponents[j] * log_scales + log_moments[j, 0]
            plt.plot(log_scales, y_fit, '--', color='gray', alpha=0.7)

        plt.xlabel('log(Δt)', fontsize=12)
        plt.ylabel('log(E[|X(t+Δt) - X(t)|^q])', fontsize=12)
        plt.title('Multifractal Scaling Analysis', fontsize=14)
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

        # Plot scaling function tau(q)
        plt.figure(figsize=(8, 6))
        plt.plot(q_values, scaling_exponents, 'o-', color='blue')
        plt.xlabel('q', fontsize=12)
        plt.ylabel('τ(q)', fontsize=12)
        plt.title('Multifractal Scaling Function τ(q)', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.show()

    return scales, scaling_exponents

def analyze_volatility_clustering(returns, max_lag=100, plot=True):
    """
    Analyze volatility clustering through autocorrelation of squared returns.

    Parameters:
    - returns: array of returns
    - max_lag: maximum lag to compute autocorrelation
    - plot: whether to plot the results

    Returns:
    - lags: lag values
    - acf: autocorrelation function values
    """
    squared_returns = returns**2

    # Compute autocorrelation of squared returns
    acf = np.zeros(max_lag)
    sr_mean = np.mean(squared_returns)
    sr_var = np.var(squared_returns)

    for lag in range(max_lag):
        if lag == 0:
            acf[lag] = 1.0
        else:
            acf[lag] = np.mean((squared_returns[:-lag] - sr_mean) *
                               (squared_returns[lag:] - sr_mean)) / sr_var

    if plot:
        plt.figure(figsize=(10, 6))
        plt.plot(range(max_lag), acf)
        plt.xlabel('Lag', fontsize=12)
        plt.ylabel('Autocorrelation', fontsize=12)
        plt.title('Autocorrelation of Squared Returns', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.axhline(y=0, color='red', linestyle='--', alpha=0.3)
        plt.show()

    return np.arange(max_lag), acf

def estimate_multifractal_spectrum(t, X, q_min=-5, q_max=5, n_q=21, plot=True):
    """
    Estimate the multifractal spectrum f(alpha) from the data.

    Parameters:
    - t: time points
    - X: log-price process
    - q_min, q_max: range of q values
    - n_q: number of q values
    - plot: whether to plot the results

    Returns:
    - alpha: Hölder exponents
    - f_alpha: multifractal spectrum
    """
    # Generate q values
    q_values = np.linspace(q_min, q_max, n_q)

    # Compute scaling exponents
    _, tau_q = analyze_scaling_properties(t, X, q_values=q_values, plot=False)

    # Compute alpha (derivative of tau(q))
    alpha = np.zeros(n_q)
    for i in range(1, n_q-1):
        # Use central difference
        alpha[i] = (tau_q[i+1] - tau_q[i-1]) / (q_values[i+1] - q_values[i-1])

    # Compute endpoint derivatives using forward/backward differences
    alpha[0] = (tau_q[1] - tau_q[0]) / (q_values[1] - q_values[0])
    alpha[-1] = (tau_q[-1] - tau_q[-2]) / (q_values[-1] - q_values[-2])

    # Compute f(alpha) through Legendre transform
    f_alpha = np.zeros(n_q)
    for i in range(n_q):
        f_alpha[i] = q_values[i] * alpha[i] - tau_q[i]

    if plot:
        plt.figure(figsize=(12, 6))

        # Plot tau(q)
        plt.subplot(1, 2, 1)
        plt.plot(q_values, tau_q, 'o-', color='blue')
        plt.xlabel('q', fontsize=12)
        plt.ylabel('τ(q)', fontsize=12)
        plt.title('Scaling Function τ(q)', fontsize=14)
        plt.grid(True, alpha=0.3)

        # Plot f(alpha)
        plt.subplot(1, 2, 2)
        plt.plot(alpha, f_alpha, 'o-', color='red')
        plt.xlabel('α', fontsize=12)
        plt.ylabel('f(α)', fontsize=12)
        plt.title('Multifractal Spectrum f(α)', fontsize=14)
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

    return alpha, f_alpha

def main():
    # Parameters
    T = 1.0                # Time horizon
    n_points = 10000       # Number of time points

    # Multifractal parameters
    k_levels = 10          # Number of cascade levels
    base_freq = 5          # Base frequency

    # Create geometric series of frequencies
    frequencies = [base_freq * 2**i for i in range(k_levels)]

    # Create a multiplier generator (lognormal with mean 1)
    multiplier_gen = lognormal_multiplier_generator(mu=-0.1, sigma=0.3)

    # Alternative: binomial multiplier
    # m0, m1 = 0.4, 1.6  # Note: m0 + m1 = 2 for canonical measure
    # multiplier_gen = binomial_multiplier_generator(m0, m1)

    # Set random seed for reproducibility
    seed = 42

    print("Simulating multifractal model...")
    t, X, theta, B = simulate_multifractal_model(
        T=T,
        n_points=n_points,
        frequencies=frequencies,
        multiplier_generator=multiplier_gen,
        seed=seed
    )

    # Get the measure (derivative of theta)
    dx = t[1] - t[0]
    measure = np.diff(theta) / dx
    x_theta = np.linspace(0, T, len(theta))

    print("Plotting model components...")
    plot_model_components(t, X, x_theta, theta, t, B, measure)

    print("Analyzing scaling properties...")
    analyze_scaling_properties(t, X, q_values=[1, 2, 3, 5], n_scales=20)

    # Compute returns
    returns = np.diff(X)

    print("Analyzing volatility clustering...")
    analyze_volatility_clustering(returns, max_lag=100)

    print("Estimating multifractal spectrum...")
    estimate_multifractal_spectrum(t, X)

    print("Done!")

if __name__ == "__main__":
    main()